In [ ]:
import tensorflow as tf
from PIL import Image 
import numpy as np
import skimage
import skimage.io
import skimage.transform
from matplotlib.pyplot import imshow
%matplotlib inline  

In [ ]:
synset = [l.strip() for l in open('synset.txt').readlines()]

def load_image(path):
    img = skimage.io.imread(path)
    imshow(Image.open(path,'r'))
    img = img / 255.0
    assert (0 <= img).all() and (img <= 1.0).all()
    #crop image from center
    short_edge = min(img.shape[:2])
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy : yy + short_edge, xx : xx + short_edge]
    resized_img = skimage.transform.resize(crop_img, (299, 299))
    return resized_img

def print_prob(prob):
    pred = np.argsort(prob)[::-1]
    top1 = synset[pred[0]-1]
    print "Top Prediction", top1
    top5 = map(synset.__getitem__, pred[:5]-1)
    print "Top 5 Prediction: ", top5


In [ ]:
with open('inception_v4.pb', 'rb') as f:
    fileContent = f.read()

graph_def = tf.GraphDef()
graph_def.ParseFromString(fileContent)
tf.import_graph_def(graph_def)
graph = tf.get_default_graph()
tensors = [n.name for n in tf.get_default_graph().as_graph_def().node]
print "graph loaded from disk"

In [ ]:
batch_size=1
def forward_pass(path):
    image = load_image(path)
    with tf.Session(graph=graph) as sess:
        batch = image.reshape((batch_size, 299, 299, 3))
        assert batch.shape == (batch_size, 299, 299, 3)
        feed_dict = {graph.get_tensor_by_name("import/InputImage:0"): batch}
        prob_tensor = graph.get_tensor_by_name("import/InceptionV4/Logits/Predictions:0")
        prob = sess.run(prob_tensor, feed_dict=feed_dict) #1001 shape vector
    print_prob(prob[0])
    return prob[0]

In [ ]:
forward_pass("/home/pranay360/Image_Captioning/Images/dog2.jpg")

In [ ]:
[  3.99079763e-05,   2.55644027e-05,   6.72903116e-05, ...,
         1.43715179e-05,   3.42611565e-05,   1.49928219e-05]
[  3.99079763e-05,   2.55644027e-05,   6.72903116e-05, ...,
         1.43715179e-05,   3.42611565e-05,   1.49928219e-05]

In [ ]:
for ten in tensors[::-1][:100]:
    print ten, graph.get_tensor_by_name(ten+":0").get_shape()